In [1]:
import pandas as pd
from datetime import datetime
from utils.paths import Paths

from performance.tools import (
    DailyReturnGenerator,
    ReturnMetricsRunner,
    MetricsInteractiveViewer,
    ReturnDataFrameBuilder,
)
from machine_learning.ml_dataset import MLDataset

## 実際のトレード履歴をリターン化

In [2]:
trade_history = pd.read_csv(Paths.TRADE_HISTORY_CSV)
drg = DailyReturnGenerator(date_series=trade_history["日付"],
                     acquisition_price_series=trade_history["取得価格"],
                     settlement_price_series=trade_history["決済価格"],
                     long_or_short_series=trade_history["売or買"],
                     short_keyphrase="売")
daily_return_df = drg.get()
daily_return_df.tail(2)

,Return
Date,
2025-07-03,0.002200
2025-07-04,-0.003571


In [3]:
rmr = ReturnMetricsRunner(date_series=daily_return_df.index,
                          return_series=daily_return_df['Return'])
df_dict = rmr.calculate()
miv = MetricsInteractiveViewer(df_dict)
miv.display()

## モデルのリターンデータ読み込み

In [12]:
#TODO ここをクラス化する

def get_return_df(pred_df: pd.DataFrame, trade_secor_numbers: int, top_slope: float) -> pd.DataFrame:
    pred_df['TargetRank'] = pred_df.groupby('Date')['Target'].rank(ascending=False)
    pred_df['PredRank'] = pred_df.groupby('Date')['Pred'].rank(ascending=False)

    sector_numbers = len(pred_df.index.get_level_values('Sector').unique())
    long_df = pred_df[pred_df['PredRank'] <= trade_secor_numbers]
    short_df = pred_df[pred_df['PredRank'] > sector_numbers - trade_secor_numbers]

    if trade_secor_numbers > 1:
        long_df.loc[long_df['PredRank'] == 1, 'Target'] *= top_slope
        long_df.loc[long_df['PredRank'] != 1, 'Target'] *= 1 - (top_slope - 1) / (trade_secor_numbers - 1)
        short_df.loc[short_df['PredRank'] == sector_numbers, 'Target'] *= top_slope
        short_df.loc[short_df['PredRank'] != sector_numbers, 'Target'] *= 1 - (top_slope - 1) / (trade_secor_numbers - 1)
    long_df = long_df.groupby('Date')[['Target']].mean()
    short_df = - short_df.groupby('Date')[['Target']].mean()
    timeseries_return = (long_df + short_df) / 2
    timeseries_return = timeseries_return.rename(columns={'Target': 'Return'})

    return timeseries_return

In [13]:
eval_start_date = datetime(2022, 1, 1)
eval_end_date = datetime.today()
trade_sector_numbers = 2
top_slope = 1

lgbm48 = f'{Paths.ML_DATASETS_FOLDER}/48sectors_LightGBMlearned_in_250702'
lgbm48_pred_df = ReturnDataFrameBuilder(dataset_root=lgbm48, start_date=eval_start_date, end_date=eval_end_date).build()
lgbm48_return_df = get_return_df(lgbm48_pred_df, trade_sector_numbers, top_slope)

lasso56 = f'{Paths.ML_DATASETS_FOLDER}/56sectors_LASSO_learned_in_250702'
lasso56_pred_df = ReturnDataFrameBuilder(dataset_root=lasso56, start_date=eval_start_date, end_date=eval_end_date).build()
lasso56_return_df = get_return_df(lasso56_pred_df, trade_sector_numbers, top_slope)

In [18]:
rmr = ReturnMetricsRunner(date_series=lasso56_return_df.index,
                          return_series=lasso56_return_df['Return'],
                          is_tax_excluded=False,
                          is_leverage_applied=False)
df_dict = rmr.calculate()
miv = MetricsInteractiveViewer(df_dict)
miv.display()

In [19]:
ensembled_return_df = (lgbm48_return_df + lasso56_return_df * 2) / 3

rmr = ReturnMetricsRunner(date_series=ensembled_return_df.index,
                          return_series=ensembled_return_df['Return'],
                          is_tax_excluded=False,
                          is_leverage_applied=False)
df_dict = rmr.calculate()
miv = MetricsInteractiveViewer(df_dict)
miv.display()